<a href="https://colab.research.google.com/github/fabriciosantana/mcdia/blob/main/01-icd/assignments/04-analise-tematica-discursos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classificação em temas dos Discursos do Senado


## Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import seaborn as sns
from datasets import load_dataset
from datasets import load_dataset_builder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)



## Carregar dados

In [ ]:
#DATA_PATH = Path('data/hf_discursos/data/full/discursos_2019-02-01_2023-01-31.parquet')
#df_raw = pd.read_parquet(DATA_PATH)

DATASET_HF_REPO = "fabriciosantana/discursos-senado-legislatura-56"
DATA_FILE_HF = {"train": "data/full/discursos_2019-02-01_2023-01-31.parquet"}

dataset = load_dataset(DATASET_HF_REPO, data_files=DATA_FILE_HF)
df_raw = dataset["train"].to_pandas()

df_raw.head()

## Preparar dados para análise temática

Filtramos colunas relevantes, padronizamos o texto e removemos discursos muito curtos para reduzir ruído.


In [ ]:
colunas = ['Data', 'NomeAutor', 'Partido', 'UF', 'TextoDiscursoIntegral']

df = (
    df_raw[colunas]
    .rename(columns={
        'TextoDiscursoIntegral': 'texto',
        'NomeAutor': 'nome_autor'
    })
    .dropna(subset=['texto'])
)

# Limpeza básica do texto e metadados
df['texto'] = df['texto'].str.strip()
df = df[df['texto'].str.len() > 0]
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')
df = df.dropna(subset=['Data'])
df['n_palavras'] = df['texto'].str.split().str.len()
df = df[df['n_palavras'] >= 30]
df = df.sort_values('Data').reset_index(drop=True)

print(f"Discursos após limpeza: {len(df):,}")
df.head()


## Selecionar amostra para acelerar modelagem

O conjunto é reduzido para acelerar a vetorização e a extração de temas sem perder representatividade temporal.


In [ ]:
MAX_DISCURSOS = 6000

if len(df) > MAX_DISCURSOS:
    df_modelo = df.sample(MAX_DISCURSOS, random_state=42).sort_values('Data').reset_index(drop=True)
else:
    df_modelo = df.copy()

print(f'Tamanho final da amostra: {len(df_modelo):,}')

df_modelo['Data'].agg(['min', 'max'])


## Configurar stop words

In [ ]:
stopwords_pt = """
a ao aos as ate com como contra da das de dela dele deles delas dentro depois desde dessa desse desta deste disso do dos e ela elas ele eles em entre essa esse essas esses esta este estou eu foi ha isso ja la mais mas mesmo mesma meus minhas na nas nem no nos nossa nosso nossas nossos ou para pela pelas pelo pelos perante pois por porque qual quando que quem se sem ser sob sobre sua suas tambem tem tendo tera teve todos todo um uma umas uns vai ver voce voces
""".split()

stopwords_pt = sorted(set(stopwords_pt))
len(stopwords_pt)


## Realizar vetorização com TF-IDF

Transformamos os discursos em vetores de características, preservando unigramas e bigramas frequentes.


In [ ]:
tfidf_vectorizer = TfidfVectorizer(
    max_features=15000,
    min_df=20,
    max_df=0.6,
    ngram_range=(1, 2),
    stop_words=stopwords_pt,
    strip_accents='unicode'
)

tfidf_matrix = tfidf_vectorizer.fit_transform(df_modelo['texto'])

print('Dimensões da matriz TF-IDF:', tfidf_matrix.shape)


## Descobrir temas com NMF

Aplicamos NMF (Non-negative Matrix Factorization) para captar padrões latentes de vocabulário que sugerem temas.


In [ ]:
N_TEMAS = 8

nmf_model = NMF(
    n_components=N_TEMAS,
    random_state=42,
    init='nndsvda',
    max_iter=400
)

W = nmf_model.fit_transform(tfidf_matrix)
H = nmf_model.components_

print('Matriz W (documentos x temas):', W.shape)
print('Matriz H (temas x termos):', H.shape)


## Analisar principais termos por tema

Listamos as palavras com maior peso em cada tema para auxiliar na interpretação semântica.


In [ ]:
def extrair_top_termos(modelo, feature_names, top_n=12):
    registros = []
    for idx, topic in enumerate(modelo.components_):
        top_indices = topic.argsort()[::-1][:top_n]
        termos = [feature_names[i] for i in top_indices]
        registros.append({
            'tema_id': idx,
            'termos': termos,
            'principais_termos': ', '.join(termos[:10])
        })
    return pd.DataFrame(registros)

feature_names = tfidf_vectorizer.get_feature_names_out()
temas_df = extrair_top_termos(nmf_model, feature_names)
temas_df['tema_label'] = temas_df['termos'].apply(lambda termos: ' / '.join(termos[:3]))

# Visualiza termos chave por tema
temas_df[['tema_id', 'tema_label', 'principais_termos']]


## Atribuir tema predominante aos discursos

Cada discurso recebe o tema com maior peso na decomposição, além da força relativa desse tema.


In [ ]:
scores_df = pd.DataFrame(W, columns=[f'tema_{i}' for i in range(N_TEMAS)])

df_temas = df_modelo.reset_index(drop=True).join(scores_df)
df_temas['tema_id'] = scores_df.values.argmax(axis=1)
df_temas = df_temas.merge(temas_df[['tema_id', 'tema_label']], on='tema_id', how='left')

soma_scores = scores_df.sum(axis=1)
df_temas['forca_tema'] = scores_df.max(axis=1) / soma_scores.replace(0, np.nan)

df_temas[['Data', 'nome_autor', 'Partido', 'tema_label', 'forca_tema']].head()


## Analisar distribuição geral de temas

Observamos quais temas aparecem com maior frequência na amostra de discursos.


In [ ]:
contagem_temas = (
    df_temas['tema_label']
        .value_counts()
        .rename_axis('tema_label')
        .reset_index(name='n_discursos')
)

plt.figure(figsize=(12, 6))
sns.barplot(data=contagem_temas, x='n_discursos', y='tema_label', hue='tema_label', palette='viridis', legend=False)
plt.title('Frequência de discursos por tema (NMF)')
plt.xlabel('Número de discursos')
plt.ylabel('Tema (palavras-chave)')
plt.tight_layout()
plt.show()

contagem_temas


## Analisar evolução temporal do interesse por tema

Acompanhamos como a participação relativa de cada tema varia mês a mês.


In [ ]:
df_temas['mes'] = df_temas['Data'].dt.to_period('M').dt.to_timestamp()
serie_temas = (
    df_temas.groupby(['mes', 'tema_label'])
        .size()
        .reset_index(name='n_discursos')
)

plt.figure(figsize=(14, 6))
sns.lineplot(data=serie_temas, x='mes', y='n_discursos', hue='tema_label', marker='o')
plt.title('Evolução mensal de discursos por tema')
plt.xlabel('Mês')
plt.ylabel('Quantidade de discursos')
plt.xticks(rotation=45)
plt.legend(title='Tema', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()

serie_temas.head()


## Analisar participação percentual dos temas

Analisamos a participação relativa de cada tema em relação ao total de discursos do mês para identificar mudanças de prioridade ao longo do tempo.


In [ ]:
serie_percentual = (
    serie_temas
        .assign(total_mes=lambda df: df.groupby('mes')['n_discursos'].transform('sum'))
        .assign(percentual=lambda df: (df['n_discursos'] / df['total_mes']) * 100)
)

plt.figure(figsize=(14, 6))
sns.lineplot(
    data=serie_percentual,
    x='mes',
    y='percentual',
    hue='tema_label',
    marker='o'
)
plt.title('Participação percentual mensal por tema')
plt.xlabel('Mês')
plt.ylabel('Percentual de discursos (%)')
plt.xticks(rotation=45)
plt.legend(title='Tema', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()

serie_percentual.head()


## Analisar percentual de discursos por tema e por mês

In [ ]:
serie_area = (
    serie_percentual
        .pivot(index='mes', columns='tema_label', values='percentual')
        .fillna(0)
        .sort_index()
)

ax = serie_area.plot.area(figsize=(14, 6), cmap='viridis', linewidth=0)
ax.set_title('Participação percentual acumulada por tema (área empilhada)')
ax.set_xlabel('Mês')
ax.set_ylabel('Percentual de discursos (%)')
ax.legend(title='Tema', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()

serie_area.tail()


## Analisar tema predominante por mês

A tabela abaixo destaca qual tema liderou o volume de discursos em cada mês e a respectiva participação percentual.


In [ ]:
tema_lider_mes = (
    serie_percentual
        .sort_values(['mes', 'percentual'], ascending=[True, False])
        .groupby('mes', as_index=False)
        .first()[['mes', 'tema_label', 'percentual']]
)

tema_lider_mes.head(12)
